In [24]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt


#data description

def describe_data():
    df = pd.read_csv('/home/dell/oasis_longitudinal.csv')
    print(df.info())
    print("Tota Rows and Columns (Rows,Columns) : ",df.shape)
#print first five rows of the dataset
    print(df.head(5))

    print(df.describe())
#data preprocessing

    df["SES"].fillna(df["SES"].median(), inplace=True)
    df["MMSE"].fillna(df["MMSE"].mean(), inplace=True)


IndentationError: unexpected indent (<ipython-input-24-83150e7b29c5>, line 22)

In [21]:
#feature selection
def count_dementiated():
    ex_df = df.loc[df['Visit'] == 1]
    ex_df['Group'] = ex_df['Group'].replace(['Converted'], ['Demented'])
    df['Group'] = df['Group'].replace(['Converted'], ['Demented'])
    sns.countplot(x='Group', data=ex_df)

#count_dementiated()

def bar_chart(feature):
    Demented = ex_df[ex_df['Group']=='Demented'][feature].value_counts()
    Nondemented = ex_df[ex_df['Group']=='Nondemented'][feature].value_counts()
    df_bar = pd.DataFrame([Demented,Nondemented])
    df_bar.index = ['Demented','Nondemented']
    df_bar.plot(kind='bar',stacked=True, figsize=(8,5))
    print(df_bar)
    
def cdr_by_gender():
    plt.figure(figsize=(10,5))
    sns.violinplot(x='M/F', y='CDR', data=df)
    plt.title('Violin plots of CDR by Gender',fontsize=14)
    plt.xlabel('Gender',fontsize=13)
    plt.ylabel('CDR',fontsize=13)
    plt.show()
    
#cdr_by_gender()

def age_by_gender():
    plt.figure(figsize=(10,5))
    sns.violinplot(x='CDR', y='Age', data=df)
    plt.title('Violin plot of Age by CDR',fontsize=14)
    plt.xlabel('CDR',fontsize=13)
    plt.ylabel('Age',fontsize=13)
    plt.show()
    
def outliers_iqr(ys):
    quartile_1, quartile_3 = np.percentile(ys, [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    return np.where((ys > upper_bound) | (ys < lower_bound))

def boxplots():
    #boxplots which shows the IQR(Interquartile Range )
    fig, axes = plt.subplots(2,3,figsize = (16,6))
    fig.suptitle("Box Plot",fontsize=14)
    sns.set_style("whitegrid")
    sns.boxplot(data=df['SES'], orient="v",width=0.4, palette="colorblind",ax = axes[0][0]);
    sns.boxplot(data=df['EDUC'], orient="v",width=0.4, palette="colorblind",ax = axes[0][1]);
    sns.boxplot(data=df['MMSE'], orient="v",width=0.4, palette="colorblind",ax = axes[0][2]);
    sns.boxplot(data=df['CDR'], orient="v",width=0.4, palette="colorblind",ax = axes[1][0]);
    sns.boxplot(data=df['eTIV'], orient="v",width=0.4, palette="colorblind",ax = axes[1][1]);
    sns.boxplot(data=df['ASF'], orient="v",width=0.4, palette="colorblind",ax = axes[1][2]);
#xlabel("Time");
#boxplots()


def plot_correlation_map( df ):
    corr = df.corr()
    _ , ax = plt.subplots( figsize =( 12 , 10 ) )
    cmap = sns.diverging_palette( 240 , 10 , as_cmap = True )
    _ = sns.heatmap(corr,cmap = cmap,square=True, cbar_kws={ 'shrink' : .9 }, ax=ax, annot = True, annot_kws = { 'fontsize' : 12 })



from sklearn import metrics
def plot_confusion_metrix(y_test,model_test):
    cm = metrics.confusion_matrix(y_test, model_test)
    plt.figure(1)
    plt.clf()
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Wistia)
    classNames = ['Nondemented','Demented']
    plt.title('Confusion Matrix')
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    tick_marks = np.arange(len(classNames))
    plt.xticks(tick_marks, classNames)
    plt.yticks(tick_marks, classNames)
    s = [['TN','FP'], ['FN', 'TP']]
    for i in range(2):
        for j in range(2):
            plt.text(j,i, str(s[i][j])+" = "+str(cm[i][j]))
    plt.show()

    
from sklearn.metrics import roc_curve, auc
def report_performance(model):

    model_test = model.predict(X_test)

    print("\n\nConfusion Matrix:")
    print("{0}".format(metrics.confusion_matrix(y_test, model_test)))
    print("\n\nClassification Report: ")
    print(metrics.classification_report(y_test, model_test))
    #cm = metrics.confusion_matrix(y_test, model_test)
    plot_confusion_metrix(y_test, model_test)


total_fpr = {}
total_tpr = {}
def roc_curves(model):
    predictions_test = model.predict(X_test)
    fpr, tpr, thresholds = roc_curve(predictions_test,y_test)
    roc_auc = auc(fpr, tpr)
    total_fpr[str((str(model).split('(')[0]))] = fpr
    total_tpr[str((str(model).split('(')[0]))] = tpr
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=1, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    plt.show()

    
    
total_accuracy = {}
def accuracy(model):
    pred = model.predict(X_test)
    accu = metrics.accuracy_score(y_test,pred)
    print("\nAcuuracy Of the Model: ",accu,"\n\n")
    total_accuracy[str((str(model).split('(')[0]))] = accu



In [22]:
def random_forest():
    rfc=RandomForestClassifier(criterion='gini',max_depth=8,max_features='auto',n_estimators=200)

    param_grid = { 
    'n_estimators': [200],
    'max_features': ['auto'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini']
    }

#CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5,scoring = 'roc_auc')
    rfc.fit(X_train, y_train.ravel())
#print("Best parameters set found on development set:")
#print(rfc.best_params_)
    report_performance(rfc) 
    roc_curves(rfc)
    accuracy(rfc)

    feat_importances = pd.Series(rfc.feature_importances_, index=feature_col_names)
    feat_importances.nlargest(8).plot(kind='barh')
    plt.title("Feature Importance:")
    plt.show()

In [23]:
def svm_model(X_train, X_test, y_train, y_test):
    svm = SVC(kernel="linear", C=0.1,random_state=0)
    svm.fit(X_train, y_train.ravel())
    report_performance(svm,X_test,y_test) 
    #roc_curves(svm)
    #accuracy(svm)